In [ ]:
import xarray as xr
from dask.distributed import Client
client = Client('tcp://10.0.0.241:8786')

In [ ]:
client

In [1]:
import os
import sys
sys.path.append(r'/efs/Lerma/scripts_aridity')
import pandas as pd
import time

from WAsheets import calculate_flux as cf
from WAsheets import hydroloop as hl

## Hydroloop run

In [2]:
wp_folder=r"/efs/Lerma/results_chirps_ssebop/netcdfs" #folder with nc files from water balance model
BASIN={
       'name': 'Mexico',
       'hydroyear':'A-DEC', #Water year end month
       'chunksize':[1, 5000, 5000],
       'unit_conversion':1e6, #1e3 for input data in MCM, 1e6 for input data in km3
       'output_folder':r"/efs/Lerma/results_chirps_ssebop/Hydroloop_results",
       'gis_data':{
               'basin_mask': r'/efs/Lerma/Static_data/Template_mexico.tif',
               'subbasin_mask':{
                       1: r"/efs/Lerma/Static_data/Template_mexico.tif"                       
                       },
               'dem':r"/efs/Lerma/Static_data/dem.tif",
               'aeisw':r"/efs/Lerma/Static_data/gmia_mexico.tif", #area equipped with surface water irrigation percentage
               'population':r"/efs/Lerma/Static_data/pop_d_mexico.tif",
               'wpl':r"/efs/Lerma/Static_data/WPL/WPL_Max1.tif",
               'environ_water_req': r"/efs/Lerma/Static_data/EWR/EWR.tif"
               },
       'data_cube':{
           'monthly':{
               'p':os.path.join(wp_folder,
                                          'Mexico_P_CHIRPS.nc'),
                'etref':os.path.join(wp_folder,
                                          'Mexico_ETref_GLEAM.nc'),
               'et':os.path.join(wp_folder,
                                          'eta_monthly.nc'),
               'i':os.path.join(wp_folder,
                                          'i_monthly.nc'),
               't':None,
               'e':None,
               'nrd':os.path.join(wp_folder,
                                          'nRD_monthly.nc'),
               'etincr':os.path.join(wp_folder,
                                          'etincr_monthly.nc'),
               'etrain':os.path.join(wp_folder,
                                          'etrain_monthly.nc'),
               'lai':os.path.join(wp_folder,
                                          'Mexico_LAI_MOD15.nc'),
              'ndm':os.path.join(wp_folder,
                                          'Mexico_NDM_ProbaV.nc'),
             'sro':os.path.join(wp_folder,
                                         'sro_monthly.nc'),
             'sroincr':os.path.join(wp_folder,
                                          'd_sro_monthly.nc'),
             'perc':os.path.join(wp_folder,
                                          'perco_monthly.nc'),
             'percincr':os.path.join(wp_folder,
                                          'd_perco_monthly.nc'),
             'bf':os.path.join(wp_folder,
                                          'bf_monthly.nc'),
            'supply':os.path.join(wp_folder,
                                          'supply_monthly.nc')
               },
           'yearly':{
                'lu':os.path.join(wp_folder,
                                          'Mexico_LU_WA_CCI.nc'),
                   }      
                     },
        'ts_data':{
                'q_in_sw':{
                        'basin':None,
                        1:None, #unit MCM
                        2:None,
                        },
                'q_in_gw':{
                        'basin':None,
                        1:None, #unit MCM
                        2:None,
                        },
                'q_in_desal':{
                        'basin':None,
                        1:None, #unit MCM
                        2:None,
                        },
                'q_outflow':{ #river flow
                        'basin':None,
                        1:None,
                        2:None
                        },
                'q_out_sw':{ #interbasin transfer
                        'basin':None,
                        1:None,
                        2:None,
                        },
                'q_out_gw':{ #interbasin transfer
                        'basin':None,
                        1:None,
                        2:None,
                        },
                'dS_sw':{
                        'basin':None,
                        1:None, #unit MCM
                        2:None,
                        },
                                
                },
        'params':{
            'crops':{#lu_code: [r'seasons_dates.csv','Crop_type']
                    35.0: [r'/efs/Cimanuk/static_datasets/growing seasons/rice_rain_java.txt','N/A'],
                    54.0: [r'/efs/Cimanuk/static_datasets/growing seasons/rice_irrigated_java.txt','N/A'],    
                    52.0: [r'/efs/Cimanuk/static_datasets/growing seasons/palm_perennial.txt','N/A'],
                    33.0: [r'/efs/Cimanuk/static_datasets/growing seasons/palm_perennial.txt','N/A']             
                    },
            'dico_in':{1:[]},
            'dico_out':{1:[]},
            'residential_sw_supply_fraction':0.6,
            'wcpc':110, #Water consumption per capita per day in [liter/person/day]
            'wcpc_min':100, #minimum demand
            'fraction_xs':[4,25,4,25]
        }
        
       }
            

In [3]:
BASIN['params']['dico_in'][1]

[]

In [4]:
start = time.time()

out_folder = BASIN['output_folder']
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [6]:
# time.sleep(10000)

In [6]:
### Resample yearly LU to monthly netCDF
yearly_nc=BASIN['data_cube']['yearly']['lu']
sample_nc=BASIN['data_cube']['monthly']['p']
monthly_nc=cf.resample_to_monthly_dataset(yearly_nc, sample_nc,
                                start_month=0,
                                output=None,
                                chunksize=BASIN['chunksize'])

Save monthly LU datacube as /efs/Lerma/results_chirps_ssebop/netcdfs/Mexico_LU_WA_CCI_resampled_monthly.nc


In [5]:

yearly_nc=BASIN['data_cube']['yearly']['lu']
sample_nc=BASIN['data_cube']['monthly']['p']
monthly_nc = r'/efs/Lerma/results_chirps_ssebop/netcdfs/Mexico_LU_WA_CCI_resampled_monthly.nc'

In [ ]:
## check this one again
BASIN['data_cube']['monthly']['lu']=monthly_nc
### Split ETI
e_nc,i_nc,t_nc=hl.split_ETI(et_nc=BASIN['data_cube']['monthly']['et'],
                            i_nc=BASIN['data_cube']['monthly']['i'],
                            t_nc=BASIN['data_cube']['monthly']['t'],
                              p_nc=BASIN['data_cube']['monthly']['p'],
                              lai_nc=BASIN['data_cube']['monthly']['lai'],
                              nrd_nc=BASIN['data_cube']['monthly']['nrd'],
                              ndm_nc=BASIN['data_cube']['monthly']['ndm']
              ) 

/home/iwmi-wa/anaconda3/envs/wa_env/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/iwmi-wa/anaconda3/envs/wa_env/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/iwmi-wa/anaconda3/envs/wa_env/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/iwmi-wa/anaconda3/envs/wa_env/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/iwmi-wa/anaconda3/envs/wa_env/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/iwmi-wa/anaconda3/envs/wa_env/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=ke

In [ ]:
BASIN['data_cube']['monthly']['i']=i_nc
BASIN['data_cube']['monthly']['t']=t_nc
BASIN['data_cube']['monthly']['e']=e_nc

In [ ]:
### split supply
sw_supply_fraction_nc=hl.calc_sw_supply_fraction_by_LU(BASIN['data_cube']['monthly']['lu'],
                                                       BASIN['gis_data']['aeisw'],
                                                      chunksize=BASIN['chunksize'])

sw_supply_nc,gw_supply_nc=hl.split_flow(BASIN['data_cube']['monthly']['supply'],
              fraction_nc=sw_supply_fraction_nc, chunksize=BASIN['chunksize'])

In [ ]:
BASIN['data_cube']['monthly']['t'] = os.path.join(wp_folder,'transpiration.nc')
BASIN['data_cube']['monthly']['e'] = os.path.join(wp_folder,'evaporation.nc')
BASIN['data_cube']['monthly']['lu'] = os.path.join(wp_folder,'AFR_LU_WA_resampled_monthly.nc')
BASIN['data_cube']['monthly']['sw_supply_fraction'] = os.path.join(wp_folder,'sw_supply_fraction.nc')
BASIN['data_cube']['monthly']['supply_sw'] = os.path.join(wp_folder,'total_sw_supply.nc')
BASIN['data_cube']['monthly']['demand'] = os.path.join(wp_folder,'total_demand.nc')
BASIN['data_cube']['monthly']['fraction'] = os.path.join(wp_folder,'fraction.nc')
BASIN['data_cube']['monthly']['return_sw'] = os.path.join(wp_folder,'return_sw.nc')
BASIN['data_cube']['monthly']['return_gw'] = os.path.join(wp_folder,'return_gw.nc')
BASIN['data_cube']['monthly']['supply'] = os.path.join(wp_folder,'total_supply.nc')
BASIN['data_cube']['monthly']['sw_supply_fraction'] = os.path.join(wp_folder,'sw_supply_fraction.nc')
BASIN['data_cube']['monthly']['sw_return_fraction'] = os.path.join(wp_folder,'sw_return_fraction.nc')
BASIN['data_cube']['monthly']['supply_monthly_sw'] = os.path.join(wp_folder,'supply_monthly_sw.nc')
BASIN['data_cube']['monthly']['supply_monthly_gw'] = os.path.join(wp_folder,'supply_monthly_gw.nc')
BASIN['data_cube']['monthly']['residential_supply'] = os.path.join(wp_folder,'residential_supply.nc')
BASIN['data_cube']['monthly']['residential_demand'] = os.path.join(wp_folder,'residential_demand.nc')

In [ ]:
# check the numbers for Irrigated crops ---- above

In [ ]:
### demand of land surface
demand_nc=hl.calc_land_surface_water_demand(BASIN['data_cube']['monthly']['lai'],
                                  BASIN['data_cube']['monthly']['etref'],
                                  BASIN['data_cube']['monthly']['p'],
                                  BASIN['data_cube']['monthly']['lu'],
                                  chunksize=BASIN['chunksize'])

In [ ]:
### non-consumed supply or return flow
return_nc=hl.substract_flow(BASIN['data_cube']['monthly']['supply'],
                                  BASIN['data_cube']['monthly']['etincr'],
                                  name='return',
                                  chunksize=BASIN['chunksize']
                                  )

In [ ]:
### split return by sroincr/total_incremental ratio
sw_return_fraction_nc=hl.calc_sw_return_fraction(
        BASIN['data_cube']['monthly']['sroincr'],
        BASIN['data_cube']['monthly']['percincr'])
sw_return_nc,gw_return_nc=hl.split_flow(return_nc,fraction_nc=sw_return_fraction_nc, chunksize=BASIN['chunksize'])

In [ ]:
### residential supply and demand
residential_supply_nc=hl.calc_residential_water_consumption(
        BASIN['gis_data']['population'],
        BASIN['gis_data']['basin_mask'],
        BASIN['data_cube']['monthly']['lu'],
        wcpc=110,
        flow_type='supply',
        chunksize=BASIN['chunksize']
        )

residential_demand_nc=hl.calc_residential_water_consumption(
        BASIN['gis_data']['population'],
        BASIN['gis_data']['basin_mask'],
        BASIN['data_cube']['monthly']['lu'],
        wcpc=100,
        flow_type='demand',
        chunksize=BASIN['chunksize'])

In [ ]:
sw_return_nc = BASIN['data_cube']['monthly']['return_sw']
gw_return_nc = BASIN['data_cube']['monthly']['return_gw']
sw_supply_fraction_nc = BASIN['data_cube']['monthly']['sw_supply_fraction']
sw_return_fraction_nc = BASIN['data_cube']['monthly']['sw_return_fraction']
residential_supply_nc = BASIN['data_cube']['monthly']['residential_supply']
residential_demand_nc = BASIN['data_cube']['monthly']['residential_demand']
sw_supply_nc = BASIN['data_cube']['monthly']['supply_monthly_sw']
gw_supply_nc = BASIN['data_cube']['monthly']['supply_monthly_gw']

In [ ]:
### split return flow by source sw/gw
return_sw_from_sw_nc,return_sw_from_gw_nc=hl.split_flow(
        sw_return_nc,fraction_nc=sw_supply_fraction_nc, chunksize=BASIN['chunksize'])
return_gw_from_sw_nc,return_gw_from_gw_nc=hl.split_flow(
        gw_return_nc,fraction_nc=sw_supply_fraction_nc, chunksize=BASIN['chunksize'])

In [ ]:
### split residential supply by sw/gw fraction
f=BASIN['params']['residential_sw_supply_fraction']
sw_residential_supply_nc,gw_residential_supply_nc=hl.split_flow(
        residential_supply_nc,fraction=f, chunksize=BASIN['chunksize'])

In [ ]:
### add residential sw/gw supply to sw/gw supply and sw/gw return
BASIN['data_cube']['monthly']['supply_sw']=hl.add_flow(
        sw_supply_nc,sw_residential_supply_nc,name='total_sw_supply', chunksize=BASIN['chunksize'])
BASIN['data_cube']['monthly']['supply_gw']=hl.add_flow(
        gw_supply_nc,gw_residential_supply_nc,name='total_gw_supply', chunksize=BASIN['chunksize'])

In [ ]:
#assume that residential supply from sw return to sw, from gw return to gw
BASIN['data_cube']['monthly']['return_sw_from_sw']=hl.add_flow(
        return_sw_from_sw_nc,sw_residential_supply_nc,name='total_return_sw_from_sw', chunksize=BASIN['chunksize'])
BASIN['data_cube']['monthly']['return_gw_from_gw']=hl.add_flow(
        return_gw_from_gw_nc,gw_residential_supply_nc,name='total_return_gw_from_gw', chunksize=BASIN['chunksize'])

BASIN['data_cube']['monthly']['return_sw_from_gw']=return_sw_from_gw_nc
BASIN['data_cube']['monthly']['return_gw_from_sw']=return_gw_from_sw_nc

In [ ]:
### add residential demand to total demand
BASIN['data_cube']['monthly']['demand']=hl.add_flow(
        demand_nc,residential_demand_nc,name='total_demand', chunksize=BASIN['chunksize'])
### total return and supply
BASIN['data_cube']['monthly']['return_sw']=hl.add_flow(
        BASIN['data_cube']['monthly']['return_sw_from_gw'],
        BASIN['data_cube']['monthly']['return_sw_from_sw'],
        name='return_sw',
        chunksize=BASIN['chunksize'])
BASIN['data_cube']['monthly']['return_gw']=hl.add_flow(
        BASIN['data_cube']['monthly']['return_gw_from_gw'],
        BASIN['data_cube']['monthly']['return_gw_from_sw'],
        name='return_gw',
        chunksize=BASIN['chunksize'])
BASIN['data_cube']['monthly']['supply']=hl.add_flow(
        BASIN['data_cube']['monthly']['supply_sw'],
        BASIN['data_cube']['monthly']['supply_gw'],
        name='total_supply',
        chunksize=BASIN['chunksize'])

In [ ]:
### calculate recharge
BASIN['data_cube']['monthly']['recharge']=BASIN['data_cube']['monthly']['perc']

BASIN['data_cube']['monthly']['fraction'] = hl.calc_fractions(BASIN['data_cube']['monthly']['p'],
                                                              dem=BASIN['gis_data']['dem'],
                                                              lu=BASIN['data_cube']['yearly']['lu'],
                                                              fraction_altitude_xs=BASIN['params']['fraction_xs'],
                                                              chunksize=BASIN['chunksize'])

In [ ]:
# Run from here

BASIN['data_cube']['monthly']['supply'] = os.path.join(wp_folder,'return.nc')
BASIN['data_cube']['monthly']['supply_sw'] = os.path.join(wp_folder,'total_sw_supply.nc')
BASIN['data_cube']['monthly']['supply_gw'] = os.path.join(wp_folder,'total_gw_supply.nc')
BASIN['data_cube']['monthly']['return_sw_from_sw'] = os.path.join(wp_folder,'total_return_sw_from_sw.nc')
BASIN['data_cube']['monthly']['return_gw_from_gw'] = os.path.join(wp_folder,'total_return_gw_from_gw.nc')
BASIN['data_cube']['monthly']['demand'] = os.path.join(wp_folder,'total_demand.nc')
BASIN['data_cube']['monthly']['return_sw'] = os.path.join(wp_folder,'return_sw.nc')
BASIN['data_cube']['monthly']['return_gw'] = os.path.join(wp_folder,'return_gw.nc')
BASIN['data_cube']['monthly']['supply'] = os.path.join(wp_folder,'total_supply.nc')
BASIN['data_cube']['monthly']['t'] = os.path.join(wp_folder,'transpiration.nc')
BASIN['data_cube']['monthly']['e'] = os.path.join(wp_folder,'evaporation.nc')
BASIN['data_cube']['monthly']['lu'] = os.path.join(wp_folder,'Incomati_LU_WA_resampled_monthly.nc')
BASIN['data_cube']['monthly']['fraction'] = os.path.join(wp_folder,'fractions.nc')

In [ ]:
### Calculate subbasin-wide timeseries

for sb in BASIN['gis_data']['subbasin_mask']:
    subbasin={}
    for key in ['sro','return_sw','bf','supply_sw']:
        output=os.path.join(BASIN['output_folder'],
                            'subbasin_{0}_{1}.csv'.format(sb,key))
        df=cf.calc_flux_per_basin(BASIN['data_cube']['monthly'][key],
                                BASIN['gis_data']['subbasin_mask'][sb],
                                output=output)
        subbasin[key]=df        
    # read subbasin inflow
    if len(BASIN['params']['dico_in'][sb])==0: #no inflow
        inflow=None 
    else: #1 or more inflows
        for i in range(len(BASIN['params']['dico_in'][sb])):            
            if BASIN['params']['dico_in'][sb][i] == 0: #inflow from outside
                needed_params = ['q_in_desal','q_in_sw','q_in_gw'] #check only inflows
                t = 0
                for q_in in BASIN['ts_data'].keys():
                    if (q_in in needed_params) and (BASIN['ts_data'][q_in][sb] != None):       #check csv                 
                        df_inflow_ = pd.read_csv(BASIN['ts_data'][q_in][sb],
                                      sep=';',index_col=0)
                        
                        if t == 0:
                            df_inflow = df_inflow_
                        else:
                            df_inflow = df_inflow + df_inflow_
                        
                        t += 1
                        
                if t == 0:                                
                    print('Warning, missing inflow textfiles, proceeding without inflow textfiles')
                 
                         
            else: #inflow from upstream subbasin                  
                subbasin_in=BASIN['params']['dico_in'][sb][i]
                df_inflow=pd.read_csv(
                        BASIN['ts_data']['q_outflow'][subbasin_in],
                                      sep=';',index_col=0) 
                # df_inflow = pd.read_csv(BASIN['ts_data']['q_in_sw'][subbasin_in],
                #                       sep=';',index_col=0)                
                #assuming that outflow of upstream subbasin was calculated before
            if i == 0:
                inflow=df_inflow
            else:
                inflow=inflow+df_inflow    
    
    ## Interbasin transfer outflow
    if BASIN['ts_data']['q_out_sw'][sb] == None:
        q_out_sw=None
    else: 
        q_out_sw = pd.read_csv(
                        BASIN['ts_data']['q_out_sw'][sb],
                                      sep=';',index_col=0)

    # calculate sw discharge and dS from pixel-based model results
    output=os.path.join(BASIN['output_folder'],
                        'subbasin_{0}_{1}.csv'.format(sb,'{0}'))
    discharge,dS_sw=hl.calc_sw_from_wp(subbasin['sro'],
                                        subbasin['return_sw'],
                                        subbasin['bf'],
                                        subbasin['supply_sw'],
                                        inflow=inflow,
                                        q_out_sw = q_out_sw,
                                        output=output,
                                        outflow=True, #not endorheic basin
                                        plot=True
                                        )
    BASIN['ts_data']['q_outflow'][sb]=discharge
    BASIN['ts_data']['dS_sw'][sb]=dS_sw    
    inflow = None

In [ ]:
# outflow of basin is outflow of downstream subbasin   
for sb in BASIN['params']['dico_out']:
    if 0 in BASIN['params']['dico_out'][sb]: #if subbasin outflow is basin outflow
        BASIN['ts_data']['q_outflow']['basin']=BASIN['ts_data']['q_outflow'][sb]

# dS_sw of basin is sum of dS_sw of all subbasins
for i in range(len(BASIN['gis_data']['subbasin_mask'])):
    sb=list(BASIN['gis_data']['subbasin_mask'].keys())[i]
    df=pd.read_csv(BASIN['ts_data']['dS_sw'][sb],sep=';',index_col=0)
    if i==0:
        dS_sw=df
    else:
        dS_sw=dS_sw+df
dS_sw.to_csv(os.path.join(BASIN['output_folder'],
                        'basin_dS_sw.csv'),sep=';')
BASIN['ts_data']['dS_sw']['basin']=os.path.join(BASIN['output_folder'],
                        'basin_dS_sw.csv')

In [ ]:
#%% yearly datacube
for key in BASIN['data_cube']['monthly']:
    if key != 'lu':
        print(key)
        BASIN['data_cube']['yearly'][key] = cf.create_yearly_dataset(
                monthly_nc = BASIN['data_cube']['monthly'][key], hydroyear = BASIN['hydroyear'], chunksize = BASIN['chunksize'])

In [ ]:
#%% Calculate yearly and (intermediate) monthly sheet csvs
from WAsheets import sheet1
from WAsheets import sheet2
from WAsheets import sheet3
from WAsheets import sheet4
from WAsheets import sheet5
from WAsheets import sheet6

sheet1_yearly_csvs=sheet1.main(BASIN,unit_conversion=BASIN['unit_conversion'])
# sheet2_yearly_csvs=sheet2.main(BASIN,unit_conversion=BASIN['unit_conversion'])

In [ ]:
from WAsheets import print_sheet as ps

if BASIN['unit_conversion'] == 1e6:
    str_unit='km3/year'
elif BASIN['unit_conversion']==1e3:
    str_unit='MCM/year'
    
for sheet1_csv in sheet1_yearly_csvs:
    period=os.path.basename(sheet1_csv).split('.')[0].split('_')[-1]
    output=sheet1_csv.replace('.csv','.pdf')
    ps.print_sheet1(BASIN['name'],period=period,
                    output=output,units=str_unit,data=sheet1_csv)


In [ ]:
from WAsheets import print_sheet as ps

if BASIN['unit_conversion'] == 1e6:
    str_unit='km3/year'
elif BASIN['unit_conversion']==1e3:
    str_unit='MCM/year'
    
for sheet2_csv in sheet2_yearly_csvs:
    period=os.path.basename(sheet2_csv).split('.')[0].split('_')[-1]
    output=sheet2_csv.replace('.csv','.pdf')
    ps.print_sheet2(BASIN['name'],period=period,
                    output=output,units=str_unit,data=sheet2_csv)

In [ ]:
end = time.time()
total_time = end - start
print ('total time :',total_time)

In [ ]:
import glob
import os
from WAsheets import print_sheet_cwa as ps
import shutil

str_unit='km3/year'

out_folder = BASIN['output_folder']

directory_path = os.path.join(out_folder,r'csv/sheet1/yearly_cwa/')
old_path = os.path.join(out_folder,r'csv/sheet1/yearly/')

try :
    shutil.copytree(old_path, directory_path)
except : 
    pass

csv_files = glob.glob(os.path.join(directory_path, '*.csv'))
sheet1_yearly_csvs = [file_path for file_path in csv_files]

basin = 'Lerma'


for sheet1_csv in sheet1_yearly_csvs:
    period=os.path.basename(sheet1_csv).split('.')[0].split('_')[-1]
    output=sheet1_csv.replace('.csv','.pdf')
    ps.print_sheet1(basin,period=period,
                    output=output,units=str_unit,data=sheet1_csv)

In [ ]:
import pandas as pd
import os
import sys
sys.path.append(r'/efs/Lerma/scripts_aridity')
from WAsheets import print_sheet_cwa as ps

# Directory containing your CSV files
directory = '/efs/Lerma/results_chirps_gleam/Hydroloop_results/csv/sheet1/yearly_cwa'

save_path = os.path.abspath(os.path.join(directory,'..','average_sheet1'))

os.makedirs(save_path) if not os.path.exists(save_path) else None

# Step 1: Read all the CSV files in the directory and extract 'VALUE' column
csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]
dataframes = []

for csv_file in csv_files:
    file_path = os.path.join(directory, csv_file)
    df = pd.read_csv(file_path, delimiter=';')
    dataframes.append(df['VALUE'])

# Step 2: Calculate the average of 'VALUE' column
average_values = sum(dataframes) / len(dataframes)

# Step 3: Choose one CSV file to replace 'VALUE' column with averaged values
# For example, we'll use the first CSV file
first_csv_file = os.path.join(directory, csv_files[0])
first_df = pd.read_csv(first_csv_file, delimiter=';')
first_df['VALUE'] = average_values

# Step 4: Save the modified CSV as a semicolon-delimited file
file_name = 'sheet1_2012-2023.csv'
average_file = os.path.join(save_path, file_name) 
first_df.to_csv(average_file, sep=';', index=False)

In [ ]:
#Run from here
import glob
import os
sys.path.append(r'/efs/Lerma/scripts_aridity')
from WAsheets import print_sheet_cwa as ps
import shutil
 
 
unit = 1e6
 
if unit == 1e6:
    str_unit='km3/year'
elif unit==1e3:
    str_unit='MCM/year'
 
# out_folder = BASIN['output_folder']
out_folder = r"/efs/Lerma/results_chirps_gleam/Hydroloop_results/csv/sheet1"
 
directory_path = os.path.join(out_folder,r'average_sheet1/')
old_path = os.path.join(out_folder,r'yearly_cwa/')
 
try :
    shutil.copytree(old_path, directory_path)
except : 
    pass
 
csv_files = glob.glob(os.path.join(directory_path, '*.csv'))
sheet1_yearly_csvs = [file_path for file_path in csv_files]
 
# basin = BASIN['name']
basin = 'Lerma'
 
 
for sheet1_csv in sheet1_yearly_csvs:
    period=os.path.basename(sheet1_csv).split('.')[0].split('_')[-1]
    output=sheet1_csv.replace('.csv','.pdf')
    ps.print_sheet1(basin,period=period,
                    output=output,units=str_unit,data=sheet1_csv)